In [ ]:
import http.client
import os
import re
import sys
import requests

from bs4 import BeautifulSoup
from getpass import getpass
from time import sleep, time as now
from urllib.parse import urljoin
from urllib.request import urlopen

In [ ]:
# Set this to 1 for debugging HTTP server communication
http.client.HTTPConnection.debuglevel = 0

In [ ]:
bamboo_base_url = "https://bamboo.ec2.local"
local_dir = os.path.expanduser('./assets/traceability')
security_certificate = os.path.expanduser("~/Americas-Root.pem")
# ec2_username = input('user: ')
# ec2_password = getpass()
ec2_username = "automatedservices"
ec2_password = "EC2RO-ResMed123+"

In [ ]:
def get_dir_name_from_url(url):
    m = re.match(r'.*/(build-\d+/Acceptance-Test-Results-\d+)/?', url)
    return m.group(1) if m else None

In [ ]:
def download_report(results_url):
    target_dir = os.path.join(local_dir, get_dir_name_from_url(results_url))
    os.makedirs(target_dir, exist_ok=True)
    url = f'{results_url}/cucumberReports/report.json'
    print(f"Downloading {url} ...", end='')

    response = requests.get(url,
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    with open(os.path.join(target_dir, 'report.json'), mode='wb') as f:
        f.write(response.content)
    print('done!')

In [ ]:
def extract_links_from_contents(contents):
    soup = BeautifulSoup(markup=contents, features="html")
    links = soup.find_all("a")
    return [
        link['href'] for link in links
        if re.match(r'.*Acceptance-Test-Results-\d+', str(link.contents))
    ]

In [ ]:
def download_reports(build_number):
    # https://bamboo.ec2.local/artifact/MON-ATEST/shared/build-2059/
    build_path = f"artifact/MON-ATEST/shared/build-{build_number}/"
    bamboo_build_url = urljoin(bamboo_base_url, build_path)
    response = requests.get(bamboo_build_url,
                            auth=(ec2_username, ec2_password),
                            verify=security_certificate)
    refs = extract_links_from_contents(response.content)
    for ref in refs:
        download_report(urljoin(bamboo_base_url, ref))

In [ ]:
download_reports(input('Build number:'))

In [ ]:
def get_local_reports(build_number):
    """Returns the file path of all the reports.json files 
    that belong to a build"""
    root = os.path.join(local_dir, f'build-{build_number}')
    result = []
    for dir_path, dir_names, file_names in os.walk(root):
        if file_names and 'report.json' in file_names:
            result.append(os.path.join(dir_path, 'report.json'))
    result.sort()
    return result

In [ ]:
get_local_reports(input('build number:'))

In [ ]:
def post_to_transform(file_name):
    transform_url = 'https://taas-traceability-service.apps.us6.paas.easycareonline.com/taas/traceability/report/transform'
    headers = {'Content-Type': 'multipart/form-data', 'Accept': 'application/json'}
    multipart_form_data = {'file': ('report.json', open(file_name, 'rb'), 'application/json', headers)}
    return requests.post(transform_url, files=multipart_form_data, verify=security_certificate)

In [ ]:
file_name = get_local_reports(2059)[0]
response = post_to_transform(file_name)
print(response.status_code)
print(response.json())

In [ ]:
!ls assets/traceability/build-2059/


In [ ]:
response.content

In [ ]:
response.json()